# Computer Vision Project

#### Approach we will take for the Aerial vs Ground Natural Disaster computer vision project

## Import Packages

In [ ]:
# Get this from GitHub Repository / Random Class Notebooks / etc.

# At minimum, we need PyTorch

# Run this cell only if working in Colab
# Connects to any needed files from GitHub and Google Drive
import os
import getpass

# Remove Colab default sample_data
!rm -r ./sample_data

# # Clone GitHub files to colab workspace
git_user = "sfhorng" # Enter user or organization name
git_email = "sh390@duke.edu" # Enter your email
repo_name = "AIPI-540-CV-Team-2-Project" # Enter repo name
# # Use the below if repo is private, or is public and you want to push to it
# # Otherwise comment next two lines out
git_token = getpass.getpass("enter git token") # Enter your github token 
git_path = f"https://{git_token}@github.com/{git_user}/{repo_name}.git"
!git clone "{git_path}"

# Install dependencies from requirements.txt file
notebook_dir = 'notebooks'
!pip install -r "{os.path.join(repo_name,'requirements.txt')}"

# Change working directory to location of notebook

path_to_data = os.path.join(repo_name,'data/raw')
%cd "{path_to_data}"
%ls

In [ ]:
import os
import urllib.request
import tarfile
import copy
import time
import numpy as np
import pandas as pd
from torchsummary import summary
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pydicom
import cv2
from PIL import Image
import zipfile

import torch
from torchvision import datasets, transforms
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

## Load data to our project

In [ ]:
# Reference the following notebooks:


# Image classification (writing neural network from scratch)

from google.colab import drive

drive.mount('/content/drive/')

# Stephanie
# path = "/content/drive/My Drive/AIPI-540-Team-2-CV-Project-Datasets/Filtered/AIDER_filtered.zip"
#Amani
path="/content/drive/My Drive/AIDER_filtered.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall(os.path.join(os.getcwd()))
zip_ref.close()

## Data Preparation: 
### Set up class to add labels, manipulate the images to size correctly, data augmentation

In [5]:
# AIDER - use approach based on image classification notebook (writing neural network from scratch)
# MEDIC - need to use DICOM

# labels = Aerial and Ground. Need to standardize "Fire", "Flood", "No Disaster" across both datasets so labels are the same

# Reference code to size images correctly. May not be in correct section, 
# but we might need to compare image sizes across our two data sets and standardize?

# Data augmentation to create new images from AIDER data set to get better class balance between aerial and ground

In [ ]:
# Set up transformations for training and validation (test) data
# For training data we will do randomized cropping to get to 224 * 224, randomized horizontal flipping, and normalization
# For test set we will do only center cropping to get to 224 * 224 and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

#train_dataset = datasets.ImageFolder(os.path.join(os.getcwd(), 'AIDER_filtered/train'), data_transforms['train'])
#val_dataset = datasets.ImageFolder(os.path.join(os.getcwd(), 'AIDER_filtered/train'), data_transforms['val'])

In [ ]:
#Create datasets for traingin and validation sets--AIDER
train_dataset = datasets.ImageFolder(os.path.join(os.getcwd(), 'AIDER_filtered/train'), data_transforms['train'])
val_dataset = datasets.ImageFolder(os.path.join(os.getcwd(), 'AIDER_filtered/train'), data_transforms['val'])

In [ ]:
# Create DataLoaders for training and validation sets
batch_size = 4
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                             shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,
                                             shuffle=False, num_workers=4)

# Set up dict for dataloaders
dataloaders = {'train':train_loader,'val':val_loader}
# Store size of training and validation sets
dataset_sizes = {'train':len(train_dataset),'val':len(val_dataset)}
# Get class names associated with labels
class_names = train_dataset.classes
#print(class_names)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Images-AIDER

In [ ]:
images, labels = iter(train_loader).next()
images = images.numpy()
fig = plt.figure(figsize=(10, 6))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size//2, idx+1, xticks=[], yticks=[])
    image = images[idx]
    image = image.transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)
    ax.imshow(image)
    ax.set_title("{}".format(class_names[labels[idx]]))

In [ ]:
#data inspection
print(f"Feature batch shape: {train_dataset.size()}")
print(f"Feature batch shape: {val_dataset.size()}")

## Data Manipulation

In [6]:
#MEDIC:
## Choose which pictures to keep from MEDIC (there are a lot more than AIDER)
## Drop pictures we don't want from MEDIC
## Drop Mild Category
## Drop Not Informative label if they exist?
## Only Keep fires, floods, and not disasters

#AIDER:
## Only keep fires, floods and not disasters


## Combine Datasets

In [7]:
# Merge MEDIC and AIDER

## Train_test_split into our training and test set

In [8]:
# Set up training, validation, and test sets

# Amani:
## Set up Model Architecture

In [17]:
# Need to decide on best architecture to use based on our goals. Performance/computational resources tradeoffs

# Are there pre-trained models that are less computationally heavy to start off with? Research this & include in PowerPoint
# Transfer Learning ^

## Train model 

In [15]:
# Reference existing code to train model

## Evaluate Performance

In [16]:
# Validation / Test set

## Report on Statistics

In [11]:
# Performance, recall, F1-score

In [12]:
# Ground vs Aerial analysis